In [1]:
import requests
import json
from bs4 import BeautifulSoup
import os
from urllib.parse import urlparse, parse_qs

In [ ]:
NEWS_URL = "https://finance.naver.com/news/news_list.naver?mode=LSS3D&section_id=101&section_id2=258&section_id3=402"
LAST_CRAWLED_FILE = "/opt/airflow/last_crawled.txt"


def fetch_article_details(url):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/125.0.0.0 Safari/537.36"
        ),
        "Referer": "https://www.google.com",
    }

    res = requests.get(url, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")

    # 대표 이미지
    image = soup.select_one('meta[property="og:image"]')["content"]

    # 기사 본문 (HTML 태그 제거한 텍스트)
    article = soup.select_one("article#dic_area").get_text(strip=True, separator="\n")

    return image, article


def convert_to_public_url(href):

    parsed = urlparse(href)
    params = parse_qs(parsed.query)
    article_id = params.get("article_id", [""])[0]
    office_id = params.get("office_id", [""])[0]
    if article_id and office_id:
        return f"https://n.news.naver.com/mnews/article/{office_id}/{article_id}"

    return href


def fetch_latest_news():
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/125.0.0.0 Safari/537.36"
        ),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Referer": "https://www.google.com/",
        "DNT": "1",  # Do Not Track
        "Upgrade-Insecure-Requests": "1",
    }

    res = requests.get(NEWS_URL, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")

    # 마지막 크롤링 시각 읽기
    last_time = None
    if os.path.exists(LAST_CRAWLED_FILE):
        with open(LAST_CRAWLED_FILE, "r") as f:
            last_time = f.read().strip()

    articles = soup.select("dl > dd.articleSummary")
    new_articles = []

    for article in articles:
        title_tag = article.find_previous_sibling("dd", class_="articleSubject").a
        title = title_tag.text.strip()
        url = convert_to_public_url(title_tag["href"])
        press = article.select_one(".press").text.strip()
        wdate = article.select_one(".wdate").text.strip()  # 예: 2025-05-22 11:34

        # 날짜 비교
        if last_time is None or wdate > last_time:
            new_articles.append(
                {"title": title, "url": url, "press": press, "wdate": wdate}
            )

    # 새 뉴스가 있다면 저장하거나 로깅
    if new_articles:
        for article in new_articles:
            # 기사 본문과 이미지를 가져옴
            image, article_text = fetch_article_details(article["url"])

            print(f"[NEW] {article['wdate']} - {article['title']} ({article['press']}) - {article['url']}")
            print(f"{article_text}\n")
        # 최신 뉴스 기준으로 last_time 갱신
        latest_time = max(article["wdate"] for article in new_articles)
        with open(LAST_CRAWLED_FILE, "w") as f:
        #     f.write(latest_time)
    else:
        print("No new articles found.")

    return new_articles

In [64]:
articles = fetch_latest_news()
articles

[NEW] 2025-05-22 14:30 - 이지스운용, 센터필드서 사랑 나눔 음악회 성료…하트하트오케스트라 초청 (매일경제) - https://n.news.naver.com/mnews/article/009/0005496926
2025년 5월20일 센터필드 야외광장에서 하트하트오케스트라가 공연을 진행하고 있다.[사진 출처=이지스자산운용]
이지스자산운용이 지난 20일 센터필드 야외 광장에서 발달장애인으로 구성된 ‘하트하트오케스트라’의 공연을 성황리에 개최했다고 22일 밝혔다.
공연은 정오부터 1시간 동안 서울 강남구 테헤란로 231에 소재한 센터필드 야외 광장에서 진행됐다.
사회공헌 프로젝트의 일환으로 진행된 이번 ‘사랑 나눔 음악회’는 음악을 통해 장애인에 대한 인식을 개선하고 나눔의 가치를 확산하기 위해 마련됐다.
하트하트오케스트라는 2006년 하트-하트재단이 창단하고, 현재 하트하트 아트앤컬처에서 운용하는 국내 대표 발달장애인 오케스트라다.
전세계 발달장애인 중 최초로 뉴욕 카네기홀, 워싱턴 D.C. 존 F.케네디 센터, 예술의전당 등 세계적인 공연장에서 공연했다.
또한 2022년 삼성호암상을 수상하고 2024년 파리 패럴림픽에서 연주하는 등 국내외에서 그 실력을 인정받고 있다.
하트하트오케스트라는 지금까지 1300여회의 국내외 연주 활동을 통해 장애인 인식 개선에 앞장서 왔다.
발달장애 당사자와 가족, 지역사회 등 전반에 긍정적인 영향을 미치며 장애인 문화예술의 새로운 지평을 열어가고 있다.
운영사인 하트하트 아트앤컬처는 지난달 설립됐다.
문화예술을 통해 발달장애인의 활발한 사회참여와 안정적인 자립을 실천하는 선도적인 장애인 문화예술 전문단체다.
이지스자산운용 관계자는 “음악적 재능을 가진 발달장애인들이 자신의 능력을 마음껏 펼칠 기회를 제공하고 동시에 많은 사람들에게 감동과 희망을 전하고자 이번 행사를 기획하게 됐다”며 “앞으로도 운용 공간을 활용해 지역사회와 함께 호흡할 수 있는 사회공헌활동을 이어가겠다”고 밝혔다.
이지스자산운용은 사회 공헌 활동 확

[{'title': '이지스운용, 센터필드서 사랑 나눔 음악회 성료…하트하트오케스트라 초청',
  'url': 'https://n.news.naver.com/mnews/article/009/0005496926',
  'press': '매일경제',
  'wdate': '2025-05-22 14:30'},
 {'title': '삼정KPMG, 기술보증기금 ‘M&A 파트너스’ 선정',
  'url': 'https://n.news.naver.com/mnews/article/009/0005496923',
  'press': '매일경제',
  'wdate': '2025-05-22 14:29'},
 {'title': '"퇴직연금, 로봇에게 맡겼더니 수익률 52%대"',
  'url': 'https://n.news.naver.com/mnews/article/014/0005353300',
  'press': '파이낸셜뉴스',
  'wdate': '2025-05-22 14:17'},
 {'title': '미래에셋증권, VIX 선물·금현물 레버리지 ETN 2종 출시',
  'url': 'https://n.news.naver.com/mnews/article/014/0005353299',
  'press': '파이낸셜뉴스',
  'wdate': '2025-05-22 14:17'},
 {'title': '미래에셋증권, VIX 선물·금현물 레버리지 ETN 2종 출시',
  'url': 'https://n.news.naver.com/mnews/article/015/0005135232',
  'press': '한국경제',
  'wdate': '2025-05-22 14:12'},
 {'title': "디피컴머설, 세계4대 기계박람회 '북경 박람회(CIMT)' 참석",
  'url': 'https://n.news.naver.com/mnews/article/014/0005353293',
  'press': '파이낸셜뉴스',
  'wdate': '2025-05-22 14:11'},
 {'title'

In [3]:
import pandas as pd

df = pd.read_csv("../../db/뉴스_종목명_포함.csv", encoding="utf-8")

In [5]:
df = df.rename(columns={"종목명": "stocks"})
df.drop(columns=["text_combined"], inplace=True)
df

,news_id,date,title,url,content,embedding,stocks
0,20221220_0001,2022-12-20,배터리도 증시 대피처 아니었다…한달 동안 -23% 빠진 LG엔솔,https://stock.mk.co.kr/news/view/9598,경기침체로 전기차 수요 부진 우려가 불거지면서 코스피 시총 2위 LG에너지솔루션의 ...,"0.454033,0.4761172,-0.21173458,0.38399482,0.03...","LG에너지솔루션, NH투자증권"
1,20221220_0002,2022-12-20,"“돈 못갚는 사람, 돈 못빌리는 사람 는다”…불황주 우울한 급등",https://stock.mk.co.kr/news/view/9617,20일 서울 중구 명동 하나은행 본점 딜링룸 모습.<사진제공=연합뉴스>\n불확실한 ...,"-0.72177464,0.21135472,0.26113853,0.4979287,0....",미래에셋증권
2,20221220_0003,2022-12-20,삼성전자 4분기 실적 ‘한파주의보’…“내년 상반기가 더 문제”,https://stock.mk.co.kr/news/view/9616,Fn가이드 기준 삼성전자의 4분기 영업이익 전망치는 7조9970억원으로 전년동기대비...,"0.1568499,0.81860757,0.897933,0.50691074,-0.17...","SK하이닉스, 삼성전자, 유진투자증권, 삼성증권, 다올투자증권"
3,20221220_0004,2022-12-20,"코리아에셋투자증권, 소외 아동 위한 ‘산타 봉사’ 진행",https://stock.mk.co.kr/news/view/9615,영등포구 지역아동센터 4곳 청소년 133명에게 ‘산타의 선물’ 증정\n코리아에셋투자...,"0.150492,0.120599695,-0.6266297,-0.06900107,-0...",NaN
4,20221220_0005,2022-12-20,‘부동산 스트레스’ 침체 걱정에 뉴욕증시 하락 마감…시장의 눈은 PCE 물가로 [월...,https://stock.mk.co.kr/news/view/9600,19일 미국 주요 지수 동반하락\nS&P 500 -1% · 나스닥 -1.5%\n12...,"-0.0045983493,0.014476272,0.75866145,0.4352570...",NaN
...,...,...,...,...,...,...,...
9000,20250518_0001,2025-05-18,“버티면 본전”…이달 테슬라·애플 몰린 서학개미 ‘눈물의 물타기’ 통했나,https://stock.mk.co.kr/news/view/749102,“단기 반등 후 추매는 신중해야”\n[이미지 = 연합뉴스]\n미국 주식에 투자하는 ...,"-0.8274727,0.42329308,0.089880824,0.6107858,0....",NaN
9001,20250518_0002,2025-05-18,“치킨값은 벌었으면”…‘몸값 8000억’ 달바글로벌 등 4곳 상장 대기,https://stock.mk.co.kr/news/view/749119,달바 미스트. [사진 = 달바 홈페이지 캡처]\n5월 넷째 주에는 8000억원의 몸...,"-0.42067158,1.505017,-0.20068085,0.30936766,-0...","유한양행, 에이피알"
9002,20250518_0003,2025-05-18,7조원 넘게 몰려들며 대박 흥행…‘승무원 미스트’로 유명한 이 기업의 상장일은,https://stock.mk.co.kr/news/view/749128,달바글로벌 코스피 입성\n서울 콘래드호텔에서 열린 달바글로벌 기업공개(IPO) 기자...,"-0.18271944,0.9524721,0.168807,-0.26182392,-0....","고려제강, 대신증권"
9003,20250518_0004,2025-05-18,'7조 흥행' 달바글로벌 22일 코스피 상장,https://stock.mk.co.kr/news/view/749142,키스트론은 일반청약 나서\n이번주(5월 19~23일) 기업공개(IPO) 시장에서는 ...,"-0.35854125,0.23000894,-0.26719934,-0.12725869...",NaN


In [6]:
df.to_csv("../../db/news_with_embedding_stocks.csv", encoding="utf-8", index=False)

In [8]:
df.stocks[0]

'LG에너지솔루션, NH투자증권'

In [9]:
import pandas as pd

df = pd.read_csv("../../db/news_with_embedding_stocks.csv", encoding="utf-8")
df

,news_id,date,title,url,content,embedding,stocks
0,20221220_0001,2022-12-20,배터리도 증시 대피처 아니었다…한달 동안 -23% 빠진 LG엔솔,https://stock.mk.co.kr/news/view/9598,경기침체로 전기차 수요 부진 우려가 불거지면서 코스피 시총 2위 LG에너지솔루션의 ...,"0.454033,0.4761172,-0.21173458,0.38399482,0.03...","LG에너지솔루션, NH투자증권"
1,20221220_0002,2022-12-20,"“돈 못갚는 사람, 돈 못빌리는 사람 는다”…불황주 우울한 급등",https://stock.mk.co.kr/news/view/9617,20일 서울 중구 명동 하나은행 본점 딜링룸 모습.<사진제공=연합뉴스>\n불확실한 ...,"-0.72177464,0.21135472,0.26113853,0.4979287,0....",미래에셋증권
2,20221220_0003,2022-12-20,삼성전자 4분기 실적 ‘한파주의보’…“내년 상반기가 더 문제”,https://stock.mk.co.kr/news/view/9616,Fn가이드 기준 삼성전자의 4분기 영업이익 전망치는 7조9970억원으로 전년동기대비...,"0.1568499,0.81860757,0.897933,0.50691074,-0.17...","SK하이닉스, 삼성전자, 유진투자증권, 삼성증권, 다올투자증권"
3,20221220_0004,2022-12-20,"코리아에셋투자증권, 소외 아동 위한 ‘산타 봉사’ 진행",https://stock.mk.co.kr/news/view/9615,영등포구 지역아동센터 4곳 청소년 133명에게 ‘산타의 선물’ 증정\n코리아에셋투자...,"0.150492,0.120599695,-0.6266297,-0.06900107,-0...",NaN
4,20221220_0005,2022-12-20,‘부동산 스트레스’ 침체 걱정에 뉴욕증시 하락 마감…시장의 눈은 PCE 물가로 [월...,https://stock.mk.co.kr/news/view/9600,19일 미국 주요 지수 동반하락\nS&P 500 -1% · 나스닥 -1.5%\n12...,"-0.0045983493,0.014476272,0.75866145,0.4352570...",NaN
...,...,...,...,...,...,...,...
9000,20250518_0001,2025-05-18,“버티면 본전”…이달 테슬라·애플 몰린 서학개미 ‘눈물의 물타기’ 통했나,https://stock.mk.co.kr/news/view/749102,“단기 반등 후 추매는 신중해야”\n[이미지 = 연합뉴스]\n미국 주식에 투자하는 ...,"-0.8274727,0.42329308,0.089880824,0.6107858,0....",NaN
9001,20250518_0002,2025-05-18,“치킨값은 벌었으면”…‘몸값 8000억’ 달바글로벌 등 4곳 상장 대기,https://stock.mk.co.kr/news/view/749119,달바 미스트. [사진 = 달바 홈페이지 캡처]\n5월 넷째 주에는 8000억원의 몸...,"-0.42067158,1.505017,-0.20068085,0.30936766,-0...","유한양행, 에이피알"
9002,20250518_0003,2025-05-18,7조원 넘게 몰려들며 대박 흥행…‘승무원 미스트’로 유명한 이 기업의 상장일은,https://stock.mk.co.kr/news/view/749128,달바글로벌 코스피 입성\n서울 콘래드호텔에서 열린 달바글로벌 기업공개(IPO) 기자...,"-0.18271944,0.9524721,0.168807,-0.26182392,-0....","고려제강, 대신증권"
9003,20250518_0004,2025-05-18,'7조 흥행' 달바글로벌 22일 코스피 상장,https://stock.mk.co.kr/news/view/749142,키스트론은 일반청약 나서\n이번주(5월 19~23일) 기업공개(IPO) 시장에서는 ...,"-0.35854125,0.23000894,-0.26719934,-0.12725869...",NaN


In [10]:
# 임베딩이 str로 되어 있다면 float list로 변환 후 다시 JSON 문자열로
def convert_embedding(val):
    try:
        float_list = [float(x) for x in val.split(",")]
        return json.dumps(float_list)
    except:
        return "[]"


# stocks도 문자열 → 리스트 → JSON 문자열
def convert_stocks(val):
    try:
        items = [x.strip() for x in val.split(",")]
        return json.dumps(items)
    except:
        return "[]"


df["embedding"] = df["embedding"].apply(convert_embedding)
df["stocks"] = df["stocks"].apply(convert_stocks)

# 저장
df.to_csv("news_with_embedding_stocks_formatted.csv", index=False)